In [1]:
import time
import config
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import requests
from difflib import SequenceMatcher
from requests_html import HTMLSession
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from tkinter import *
from tkinter import ttk
from tkinter import messagebox as mb
import tkinter as tk


In [158]:
class palace_bot:
    
    def __init__(self):
        self.base_url = base_url = 'https://shop.palaceskateboards.com'
        self.new_items_url = f'{base_url}/collections/new'
        self.desired_item = 'PALASKA KNIT'
        self.desired_item_colour = 'CREAM'
        self.size = 'Medium'
        self.basket = []
        self.item_urls = []
        self.season_range_items_name = []
        self.season_range_items = []
        self.size_list = []

    def find_products_in_new(self):
        session = HTMLSession() #TODO keep products refreshing if the if statement isnt satisfied for any product
        response = session.get(self.new_items_url)
        products = response.html.find('.product-grid-item.clearfix')
        print(len(products))
        for product in products:
            product_name = product.find('.title', first=True).text
            print(product_name)
            if SequenceMatcher(None, " ".join(product_name.split()[:-1]), self.desired_item.get()).ratio() > 0.7:# and product_name.split()[-1] == self.desired_item_colour:
                self.basket.append(product)
                
        self.fetch_desired_item_urls_from_basket()
                
    def fetch_desired_item_urls_from_basket(self):
        for product in self.basket:
            product_url = self.basket[0].find('.product-link', first=True)
            product_url_formatted = product_url.xpath('//a/@href', first=True)
            self.item_urls.append(f'{self.base_url}{product_url_formatted}')
            
    def execute_selenium_bot(self):
        #Open up the browser, adjust options to hide the botness --> Adding cookies will also be good
        option = webdriver.ChromeOptions()
        option.add_argument('--disable-blink-features=AutomationControlled')
        option.add_argument('start-maximized')
        option.add_experimental_option("excludeSwitches", ["enable-automation"])
        option.add_experimental_option('useAutomationExtension', False)
        agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"
        option.add_argument(f"user-agent={agent}")
        driver = webdriver.Chrome(executable_path="E:/Downloads/chromedriver.exe", chrome_options=option)
        driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

        #Get the first item URL
        driver.get(self.item_urls[0])
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f"//select[@name='id']/option[text()='{self.size}']"))).click()
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//input[@type='submit' and @value='Add to Cart']"))).click()
        
        #Go to cart --> Takes a split second for items to appear
        time.sleep(.5)
        driver.get(f'{self.base_url}/cart')

        #Agree with T&Cs and proceed
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//input[@id='terms-checkbox']")))
        driver.execute_script("arguments[0].click();", driver.find_element_by_xpath("//input[@id='terms-checkbox']"))
        driver.find_element_by_id('checkout').click()

        #Delivery info
        driver.find_element_by_id('checkout_email').send_keys("test@test.com")
        driver.find_element_by_id('checkout_shipping_address_first_name').send_keys("John")
        driver.find_element_by_id('checkout_shipping_address_last_name').send_keys("Doe")
        driver.find_element_by_id('checkout_shipping_address_address1').send_keys("1 Palace Road")
        driver.find_element_by_id('checkout_shipping_address_city').send_keys("London")
        driver.find_element_by_id('checkout_shipping_address_zip').send_keys("BR1 1AA")
        driver.find_element_by_id('checkout_shipping_address_phone').send_keys("0123456789")

        #Click on captcha
        iframes = driver.find_elements_by_tag_name("iframe")
        driver.switch_to.frame(iframes[1])
        driver.find_element_by_id('recaptcha-anchor').click()

        time.sleep(5) #Captcha solving time (want to get rid off this)
        driver.switch_to.default_content()
        driver.find_element_by_id('continue_button').click()

        driver.find_element_by_id('continue_button').click()

        time.sleep(2) #Having to sleep because code below isn't working
        # WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//input[@type=submit and @id='continue_button']")))

        #Card information    
        iframes = driver.find_elements_by_tag_name("iframe")
        driver.switch_to.frame(iframes[1])
        driver.find_element_by_id('number').send_keys("1234 1234 1234")
        driver.switch_to.default_content()
        driver.switch_to.frame(iframes[2])
        driver.find_element_by_id('name').send_keys("MR JOHN DOE")
        driver.switch_to.default_content()
        driver.switch_to.frame(iframes[4])
        driver.find_element_by_id('expiry').send_keys("01")
        driver.find_element_by_id('expiry').send_keys("25")
        driver.switch_to.default_content()
        driver.switch_to.frame(iframes[5])
        driver.find_element_by_id('verification_value').send_keys("123")

        #Complete purchase
        driver.switch_to.default_content()
#         driver.find_element_by_id('continue_button').click()

    def find_all_clothes(self):

        range_url = 'https://www.palaceskateboards.com/range/ultimo-2020/'
        season_range_items_name = []
        season_range_items = []
        session = HTMLSession()
        response = session.get(range_url)
        products = response.html.find('.collection.clearfix')

        for product in products:
            product_name = product.find('.title', first=True).text
            self.season_range_items_name.append(product_name)
            self.season_range_items.append(product)
        
            
    def find_all_colours(self):
        self.find_products_in_new()
        global clothes_entry
        global colour_list
        session = HTMLSession()
        
        self.find_all_clothes()

        colour_list = []
#         print(self.season_range_items)
        for product in self.season_range_items:
            if product.find('.title', first=True).text == self.desired_item.get():
                product_url = product.find('a', first=True).attrs['href']
                response = session.get(product_url)
                names = response.html.find('.slide')
                for name in names:
                    colour = str(name).split('data-name=\'')[1].split('\'')[0].split()[-1] #god forgive me
                    colour_list.append(colour)
        colour_list = set(colour_list)
#         self.get_sizes(self.item_urls[0])
        self.refresh()
        return colour_list
    
    def get_sizes(self, url): #THIS IS POINTLESS
        session = HTMLSession()
        response = session.get(url)
        self.size_list = response.html.find('.select-wrapper', first=True).text.split()
        print(self.size_list)

    def refresh(self):
        global colour_entry_menu
        global size_entry_menu
        global colour_entry
        
        menu = colour_entry_menu["menu"]
        menu.delete(0, END)
        for colour in colour_list:
                menu.add_command(label=colour, command=lambda value=colour:self.desired_item_colour.set(value))
        self.desired_item_colour.set(None)

#         menu = size_entry_menu["menu"]
#         menu.delete(0, END)
#         for size in self.size_list:
#                 menu.add_command(label=size, command=lambda value=size:self.size.set(value))
#         self.size.set(None)

                

In [136]:
str(pb.basket[0].find('a')[0]).split('href=\'')[1][:-2]

'https://www.palaceskateboards.com/range/ultimo-2020/bossy-jacket/'

In [159]:
pb = palace_bot()
# pb.find_products_in_new()
# pb.fetch_desired_item_urls_from_basket()

# pb.create_gui()
# pb.execute_selenium_bot()

window = Tk()
global colour_list
global clothes_entry
global size_list
global colour_entry_menu
global size_entry_menu
size_list = ['Small', 'Medium', 'Large', 'X-Large'] #Make this adaptive
colour_list=[None]

pb.find_all_clothes()


window.title("Palace Copper 1.0")
window.geometry("500x200")
text = Label(window, text="Palace Copper 1.0")
text.pack()

item_selection = Label(window, text='Select an ITEM:')
item_selection.place(x=0, y=38)
# clothes_entry = StringVar(window)
pb.desired_item = StringVar(window)
pb.desired_item.set(None)
clothes_entry_menu = OptionMenu(window, pb.desired_item, *pb.season_range_items_name)
clothes_entry_menu.place(x=100, y=34)
item_selection_button = Button(window, text="Enter", command=pb.find_all_colours)
item_selection_button.place(x=350, y=36)

colour_selection = Label(window, text='Select a COLOUR:')
colour_selection.place(x=0, y=80)
pb.desired_item_colour = StringVar(window)
pb.desired_item_colour.set(None)
colour_entry_menu = OptionMenu(window, pb.desired_item_colour, *colour_list)
colour_entry_menu.place(x=100, y=76)

size_selection = Label(window, text='Select a SIZE:')
size_selection.place(x=0, y=122)
pb.size = StringVar(window)
pb.size.set(None)
size_entry_menu = OptionMenu(window, pb.size, *size_list)
size_entry_menu.place(x=100, y=118)

cop_button = Button(window, text="COP")
cop_button.config( height =2, width=68)
cop_button.place(x=5, y=155)

window.mainloop()

30
ONE 2 ONE REVERSIBLE JACKET NAVY
ONE 2 ONE REVERSIBLE JACKET BROWN
ONE 2 ONE REVERSIBLE JACKET BLACK
PALASKA KNIT CREAM
PALASKA KNIT BLACK
PALASKA KNIT BLUE
POLARTEC LAZER HOOD FLURO GREEN
POLARTEC LAZER HOOD MULTI
POLARTEC LAZER HOOD RED
POLARTEC LAZER HOOD EBONY
POLARTEC LAZER HOOD WHITE
POLARTEC LAZER HOOD KHAKI
POLARTEC LAZER JOGGERS FLURO GREEN
POLARTEC LAZER JOGGERS MULTI
POLARTEC LAZER JOGGERS RED
POLARTEC LAZER JOGGERS EBONY
POLARTEC LAZER JOGGERS WHITE
POLARTEC LAZER JOGGERS KHAKI
P-FLAMES HOOD PURPLE
P-FLAMES HOOD GREY MARL
P-FLAMES HOOD BLACK
P-FLAMES HOOD WHITE
P-FLAMES HOOD CAMO
SLOW YOUR ROLL JUMPER BLACK
SLOW YOUR ROLL JUMPER YELLOW
SLOW YOUR ROLL JUMPER GREEN
SLOW YOUR ROLL JUMPER BLUE
THE STRIPE IS RIGHT T-SHIRT NAVY
THE STRIPE IS RIGHT T-SHIRT OFF WHITE
THE STRIPE IS RIGHT T-SHIRT ORANGE


[]

In [23]:
def find_all_clothes():

    range_url = 'https://www.palaceskateboards.com/range/ultimo-2020/'
    season_range_items_name = []
    season_range_items = []
    session = HTMLSession()
    response = session.get(range_url)
    products = response.html.find('.collection.clearfix')
    
    for product in products:
        product_name = product.find('.title', first=True).text
        season_range_items_name.append(product_name)
        season_range_items.append(product)
        
    return season_range_items_name, season_range_items

def find_all_colours():
    global clothes_entry
    global colour_list
    session = HTMLSession()

    
    products, products_object = find_all_clothes()
    
    colour_list = []
    for product in products_object:
        if product.find('.title', first=True).text == clothes_entry.get():
            product_url = product.find('a', first=True).attrs['href']
            response = session.get(product_url)
            names = response.html.find('.slide')
            for name in names:
                colour = str(name).split('data-name=\'')[1].split('\'')[0].split()[-1] #god forgive me
                colour_list.append(colour)
    colour_list = set(colour_list)
    get_sizes(product_url)
    refresh()
    return colour_list

def get_sizes(url):
    
    session = HTMLSession()
    response = session.get(url)
    size_list = response.html.find('.select-wrapper', first=True).text.split()

def refresh():
    #Unselct the option
    menu = colour_entry_menu["menu"]
    menu.delete(0, END)
    for colour in colour_list:
            menu.add_command(label=colour, 
                             command=lambda value=colour:
                                  colour_entry.set(value))
    colour_entry.set(None)
    
    menu = size_entry_menu["menu"]
    menu.delete(0, END)
    for size in size_list:
            menu.add_command(label=size, 
                             command=lambda value=size:
                                  size_entry.set(value))
    

In [251]:


window = Tk()
global colour_list
global clothes_entry
global size_list
size_list = [None] #Make this adaptive
colour_list=[None]

season_range_items = find_all_clothes()[0]

window.title("Palace Copper 1.0")
window.geometry("500x200")
text = Label(window, text="Palace Copper 1.0")
text.pack()

item_selection = Label(window, text='Select an ITEM:')
item_selection.place(x=0, y=38)
clothes_entry = StringVar(window)
clothes_entry.set(None)
clothes_entry_menu = OptionMenu(window, clothes_entry, *season_range_items)
clothes_entry_menu.place(x=100, y=34)
item_selection_button = Button(window, text="Enter", command=find_all_colours)
item_selection_button.place(x=350, y=36)

colour_selection = Label(window, text='Select a COLOUR:')
colour_selection.place(x=0, y=80)
colour_entry = StringVar(window)
colour_entry.set(None)
colour_entry_menu = OptionMenu(window, colour_entry, *colour_list)
colour_entry_menu.place(x=100, y=76)
# colour_entry_button = Button(window, text="Enter")
# colour_entry_button.place(x=350, y=78)

size_selection = Label(window, text='Select a SIZE:')
size_selection.place(x=0, y=122)
size_entry = StringVar(window)
size_entry.set(None)
size_entry_menu = OptionMenu(window, size_entry, *size_list)
size_entry_menu.place(x=100, y=118)
# size_entry_button = Button(window, text="Enter")
# size_entry_button.place(x=350, y=120)

cop_button = Button(window, text="COP")
cop_button.config( height =2, width=68)
cop_button.place(x=5, y=155)

window.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "E:\Python\Anaconda\lib\tkinter\__init__.py", line 1702, in __call__
    return self.func(*args)
  File "<ipython-input-210-8845f845beb1>", line 35, in find_all_colours
    get_sizes(product_url)
  File "<ipython-input-210-8845f845beb1>", line 43, in get_sizes
    size_list = response.html.find('.select-wrapper', first=True).text.split()
AttributeError: 'NoneType' object has no attribute 'text'
